In order to run this, you should have vector databases `(vectorDB)` already stored as .zip. You can download it using the link
https://github.com/dheer183/Capstone-AI-Service-Bot/tree/1c3706ccd96bd8a637e9d1ac0d29006672da5deb/Service/vectorDB

In [ ]:
!pip install chromadb langchain-chroma langchain langchain-community langchain-text-splitters langchain-groq langchain-huggingface --force-reinstall numpy==1.26.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 85.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.8 MB/s eta

Run the code snippet below it has every library included.

In [ ]:
import os
import pandas as pd
from transformers import pipeline

# Langchain modules
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"


# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L12-v2')
# embeddings = HuggingFaceEmbeddings() # Make sure you use the same embedders as you used to embed the original dataset into vector database.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Unziping

In [ ]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")

Extracting /content/service_data_vectorDB.zip...
Extraction completed for service_data_vectorDB.zip!
Extracting /content/parts_data_vectorDB.zip...
Extraction completed for parts_data_vectorDB.zip!


## Reading the vector database

In [ ]:
import os
from langchain.vectorstores import Chroma

# Directory containing vector databases
vector_db_dir = "/content/content/"
embedding_function = embeddings  # Ensure this is defined earlier in your code

# Initialize a list to store retrievers
retrievers = []

# Iterate through all subdirectories in the specified directory
for sub_dir in os.listdir(vector_db_dir):
    full_path = os.path.join(vector_db_dir, sub_dir)
    if os.path.isdir(full_path):  # Check if it's a directory
        print(f"Loading vector database from: {full_path}")

        # Load the vector database
        vectordb = Chroma(
            persist_directory=full_path,
            embedding_function=embedding_function
        )

        # Create a retriever and add it to the list
        retrievers.append(vectordb.as_retriever())

# Define a combined retriever function
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        # Retrieve documents from each retriever
        combined_results.extend(retriever.invoke(query))

    return combined_results

# Example Usage
query = "What is the price of a fuel injector for 2005 toyota rav 4 ?"
results = combined_retriever(query)
print(f"Retrieved {len(results)} documents.")

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")

Loading vector database from: /content/content/parts_data
Loading vector database from: /content/content/service_data
Retrieved 8 documents.


## Chatbot finetuning and chatting

In [ ]:
# Create a prompt template
chatbot_prompt = """
Automotive Assistant Protocol
Always follow these steps in order:

Vehicle Identification

If ANY of these are missing, ask immediately:
[Make], [Model], [Year], [Engine Size]

Example: "Please confirm your vehicle's engine size (e.g., 2.0L Turbo)."

Pricing Components

Parts: Mid-range OEM-equivalent only (always state brand + price)
Example: "Bosch 02+ Oxygen Sensor - $85"

Labor: (Hours × $130). If data missing:
"Labor estimate unavailable - consult local shop"

Fixed Services:
• Oil Change: $100 (5L oil + filter + labor)
• Tire Repair: $50 (puncture fix + balance)

Mandatory Checks

Engine Size Gate: For engine-dependent parts (filters, belts, pumps):
"Need engine size to continue (e.g., 3.5L V6)."

Year Handling: If part unavailable for requested year:
"No [Part] for 2015 - using 2012 version at $X."

Response Template

Copy
Confirming [Year] [Make] [Model] [Engine Size]:

[Service] Estimate:
• Part: [Brand/Name] - $X
• Labor: [Y] hrs × $130 = $Z
**TOTAL: $(X+Z)**

[For Oil Changes]:
Includes:
- 5W-30 Full Synthetic (5L)
- WIX XP Filter
- Labor & disposal

Related Services:
• [Fluid/Subpart 1] - $

Rules

Always state: "Mid-range [Brand] selected for quality/value balance"

Never exceed 3 related items

If complex repair: "Professional installation strongly recommended"

Token limit: Strict 1,000 characters
Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.invoke(
            {"input_documents": combined_results, "question": prompt}, return_only_outputs=True # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response['output_text']}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: 2000 Toyota Rav 4 1.8L Brakes


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1147 > 1024). Running this sequence through the model will result in indexing errors



 **SBG**: Confirming 2000 Toyota RAV4 1.8L: 
Brake Pad Replacement Estimate:
• Part: Mid-range Bosch selected for quality/value balance, Bosch Brake Pads - $45
• Labor: 1.8 hrs × $130 = $234
**TOTAL: $279**

Related Services:
• Brake Fluid Check - $20 
• Rotor Resurfacing - $75

 **You**: exit
Chatbot: Goodbye!


In [ ]:
!pip install streamlit pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.1 MB/s eta 0:00:00


In [34]:
%%bash
cat > app.py << 'EOF'
# app.py

import os
import pandas as pd
import streamlit as st
from transformers import pipeline

# LangChain modules
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma  # Use this version of Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.llms import HuggingFacePipeline
from langchain.chains.question_answering import load_qa_chain
from langchain_groq import ChatGroq
import nest_asyncio
nest_asyncio.apply()

# --------------------------
# SET API KEYS & EMBEDDINGS
# --------------------------

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"

# Initialize embeddings using the Sentence Transformers model from Hugging Face
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2')
embedding_function = embeddings  # This will be used in vector database initialization

# --------------------------
# VECTOR DATABASE INITIALIZATION
# --------------------------

# Directory containing vector databases (ensure this directory exists in your Colab environment)
vector_db_dir = "/content/content/"

# Initialize a list to store retrievers
retrievers = []

if os.path.exists(vector_db_dir):
    for sub_dir in os.listdir(vector_db_dir):
        full_path = os.path.join(vector_db_dir, sub_dir)
        if os.path.isdir(full_path):
            st.write(f"Loading vector database from: {full_path}")
            # Load the vector database using Chroma and the specified embedding function
            vectordb = Chroma(
                persist_directory=full_path,
                embedding_function=embedding_function
            )
            # Add the retriever for this database to the list
            retrievers.append(vectordb.as_retriever())
else:
    st.error(f"Vector database directory '{vector_db_dir}' not found!")

# Define a combined retriever that queries all vector databases
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        combined_results.extend(retriever.invoke(query))
    return combined_results

# --------------------------
# LLM & QA CHAIN INITIALIZATION
# --------------------------

# Initialize the LLM using ChatGroq
llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

# Load a question-answering chain using the "map_reduce" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")

# --------------------------
# CHATBOT PROMPT TEMPLATE
# --------------------------

chatbot_prompt = """
Automotive Assistant Protocol
Always follow these steps in order:

Vehicle Identification

If ANY of these are missing, ask immediately:
[Make], [Model], [Year], [Engine Size]

Example: "Please confirm your vehicle's engine size (e.g., 2.0L Turbo)."

Pricing Components

Parts: Mid-range OEM-equivalent only (always state brand + price)
Example: "Bosch 02+ Oxygen Sensor - $85"

Labor: (Hours × $130). If data missing:
"Labor estimate unavailable - consult local shop"

Fixed Services:
• Oil Change: $100 (5L oil + filter + labor)
• Tire Repair: $50 (puncture fix + balance)

Mandatory Checks

Engine Size Gate: For engine-dependent parts (filters, belts, pumps):
"Need engine size to continue (e.g., 3.5L V6)."

Year Handling: If part unavailable for requested year:
"No [Part] for 2015 - using 2012 version at $X."

Response Template

Confirming [Year] [Make] [Model] [Engine Size]:

[Service] Estimate:
• Part: [Brand/Name] - $X
• Labor: [Y] hrs × $130 = $Z
**TOTAL: $(X+Z)**

[For Oil Changes]:
Includes:
- 5W-30 Full Synthetic (5L)
- WIX XP Filter
- Labor & disposal

Related Services:
• [Fluid/Subpart 1] - $

Rules

Always state: "Mid-range [Brand] selected for quality/value balance"

Never exceed 3 related items

If complex repair: "Professional installation strongly recommended"

Token limit: Strict 1,000 characters
Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# --------------------------
# STREAMLIT CHATBOT INTERFACE
# --------------------------

# Initialize session state for chat history
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []  # Stores dicts with keys "role" and "content"

st.title("Car Issue Chatbot")
st.markdown("Welcome to the Car Issue Chatbot! Type your message below and press **Send**. Type 'exit' to clear the conversation.")

# Text input for new query
user_query = st.text_input("You:")

if st.button("Send") and user_query:
    # Check for exit command
    if user_query.strip().lower() == "exit":
        st.session_state.chat_history = []
        st.success("Conversation cleared!")
    else:
        # Append user query to the chat history
        st.session_state.chat_history.append({"role": "User", "content": user_query})

        # Create a conversation history string
        history_text = "\n".join([f"{msg['role']}: {msg['content']}" for msg in st.session_state.chat_history])
        prompt = chatbot_prompt.format(
            chat_history=history_text,
            user_input=user_query
        )

        # Retrieve documents using the combined retriever
        combined_results = combined_retriever(user_query)

        # Run the QA chain with the retrieved documents and prompt
        with st.spinner("Processing your query..."):
            response = qa_chain.invoke(
                {"input_documents": combined_results, "question": prompt},
                return_only_outputs=True
            )

        # Extract the response text
        answer_text = response.get("output_text", "No response obtained.")

        # Append assistant's response to the chat history
        st.session_state.chat_history.append({"role": "Assistant", "content": answer_text})

# Display the conversation history
st.markdown("### Conversation")
for message in st.session_state.chat_history:
    if message["role"] == "User":
        st.markdown(f"**You:** {message['content']}")
    else:
        st.markdown(f"**Assistant:** {message['content']}")

EOF


In [35]:
!ngrok authtoken 2uETp8xUJY6tnPEkNNwlFeFfwG8_7oMqjwPtxBBXV1NPbJ3bJ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [36]:
from pyngrok import ngrok
import os

# Kill any process already running on port 8501
!fuser -k 8501/tcp

# Open a tunnel on port 8501 (using "addr" instead of "port")
public_url = ngrok.connect(addr='8501')
print("Streamlit app available at:", public_url)

# Run the Streamlit app in the background
!streamlit run app.py -- --server.folderWatchBlacklist "torch.classes"


Streamlit app available at: NgrokTunnel: "https://ad9f-34-106-208-138.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.106.208.138:8501

2025-03-24 18:27:39.834068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742840859.898167   46201 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742840859.917993   46201 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/content/app.py:47: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated versio